# Regression Basics

$\mathbf{Y} = \mathbf{X}\mathbf{\beta} + \mathbf{\epsilon}$

- All of the above are matrices
- $\mathbf{X}$ includes a const column

## Setup and Load Data

In [27]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

data = load_boston()
X, y = load_boston(return_X_y=True)

X = pd.DataFrame(data=X, columns=data['feature_names'])
X['const'] = 1

y = pd.Series(data=y, name='price')

print(data['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [19]:
X.shape

(506, 14)

In [20]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,const
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,1
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,1
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,1
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,1
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,1


In [21]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: price, dtype: float64

# Ordinary Least Squares

- https://en.wikipedia.org/wiki/Ordinary_least_squares
- https://www.stat.purdue.edu/~boli/stat512/lectures/topic3.pdf

$
\mathbf{Y} = \mathbf{X}\mathbf{\beta} + \epsilon
$

We want to minimize the error term ($\mathbf{\epsilon}$), specifically the square error: $\epsilon^T \epsilon$

Why square error? We need something that looks at the magniture of error and this ends up providing nice properties.

1. $\epsilon = \mathbf{y} - \mathbf{X}\mathbf{\beta}$
1. $\epsilon^T \epsilon = (\mathbf{y} - \mathbf{X}\mathbf{\beta})^T (\mathbf{y} - \mathbf{X}\mathbf{\beta})$

Since we wish to minimize the error we can take the derivative and find the minimum point when the derivative is $0$.

1. $\frac{d}{d\beta}(\mathbf{y} - \mathbf{X}\mathbf{\beta})^T (\mathbf{y} - \mathbf{X}\mathbf{\beta})$
1. $-2 \mathbf{X}^T (\mathbf{y} - \mathbf{X}\mathbf{\beta})$

Now to find a minimum we set this derviative equal to $0$ and solve for $\beta$

1. $0 = -2 \mathbf{X}^T (\mathbf{y} - \mathbf{X}\mathbf{\beta})$
1. $0 = -2 \mathbf{X}^T \mathbf{y} + 2 \mathbf{X}^T \mathbf{X} \mathbf{\beta}$
1. $\mathbf{X}^T \mathbf{X} \mathbf{\beta} = \mathbf{X}^T \mathbf{y}$
1. $\mathbf{\beta} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}$

There are a few assumptions at play here to make this work. Only with these assumptions in play will there be a valid minimum.

- $(\mathbf{X}^T \mathbf{X})$ needs to be invertible which it won't be if you don't have enough independent samples (full rank)
- TODO other assumptions

In [51]:
# Let's try it in python

sum_mult_ftrs = (X.T.dot(X))
inv = np.linalg.inv(sum_mult_ftrs)
beta = inv.dot(X.T).dot(y)

beta = pd.DataFrame(data=beta, index=X.columns)
beta

,0
CRIM,-0.108011
ZN,0.046420
INDUS,0.020559
CHAS,2.686734
NOX,-17.766611
RM,3.809865
AGE,0.000692
DIS,-1.475567
RAD,0.306049
TAX,-0.012335


In [47]:
# Let's compare to statsmodels!
import statsmodels.api as sm

ols = sm.OLS(y, X)
results = ols.fit()
results.params.to_frame()

,0
CRIM,-0.108011
ZN,0.046420
INDUS,0.020559
CHAS,2.686734
NOX,-17.766611
RM,3.809865
AGE,0.000692
DIS,-1.475567
RAD,0.306049
TAX,-0.012335


# Where this breaks!

In [50]:
# Not enough data (too small rank)
X_small = X.head()

sum_mult_ftrs = (X_small.T.dot(X_small))
np.linalg.inv(sum_mult_ftrs)

LinAlgError: Singular matrix

In [61]:
# Not enough data (too small rank)
X_small = np.repeat(X.iloc[0, :].values.reshape(1, -1), 25, axis=0)

sum_mult_ftrs = (X_small.T.dot(X_small))
np.linalg.inv(sum_mult_ftrs)

LinAlgError: Singular matrix

In [65]:
# Not enough data (too small rank)
X_small = X
X_small['rep'] = X.iloc[:, 0]

sum_mult_ftrs = (X_small.T.dot(X_small))
inv = np.linalg.inv(sum_mult_ftrs)
beta = inv.dot(X_small.T).dot(y)

beta = pd.DataFrame(data=beta, index=X.columns)
beta

,0
CRIM,-0.951694
ZN,0.046420
INDUS,0.020559
CHAS,2.686734
NOX,-17.766611
RM,3.809865
AGE,0.000692
DIS,-1.475567
RAD,0.306049
TAX,-0.012335


In [66]:
-0.108011

-0.951694 + 0.756903

-0.19479100000000005

In [69]:
np.linalg.matrix_rank(X_small), X_small.shape

(14, (506, 15))

In [ ]:
# 

# Computing Beta p values and confidence intervals

# Regularization!

## Ridge Is Closed Form

https://en.wikipedia.org/wiki/Tikhonov_regularization

Now we add $\sum \beta _{i}^{2}$ to the cost. In matrix form: $\beta^T \beta$